# 使用 LSTM 来自动生成周杰伦歌词的简单示范

在这一个章节中，我们介绍使用 PyTorch 来处理序列化的数据，比如句子。

我们直接使用 nn.LSTM 来模拟生成周杰伦歌词。

对于 RNN 理解可以参考前一篇 [RNN](https://github.com/LianHaiMiao/pytorch-lesson-zh/blob/master/basis/rnn.ipynb)

PS: 数据来源：[李沐-GLUON-循环神经网络 — 从0开始](https://zh.gluon.ai/chapter_recurrent-neural-networks/rnn-scratch.html)

## RNN

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

In [2]:
class Corpus(object):
    """
        构建语料库的类
        path: 文件路径
    """
    def __init__(self, path):
        self.path = path
        self.char2id = {}
        self.id2char = {}
        self.corpus_indices = None
    def get_data(self):
        with open(self.path, 'r', encoding='utf8') as f:
            chars = f.read()
        chars_list = chars.replace('\n', ' ').replace('\r', ' ')
        # 开始创建索引 word 2 id
        idx = 0
        for char in chars_list:
            if not char in self.char2id:
                self.char2id[char] = idx
                self.id2char[idx] = char
                idx += 1
        # 将 corpus 里面的 char 用 index表示
        self.corpus_indices = [self.char2id[char] for char in chars_list]
    
    # 获取 corpus 的长度
    def __len__(self):
        return len(self.char2id)

In [3]:
# 构建 Config 类，用于控制超参数
class Config(object):
    def __init__(self):
        self.embed_size = 128 # embedding size
        self.hidden_size = 1024 # RNN中隐含层的 size
        self.num_layers = 1 # RNN 中的隐含层有几层，我们默认设置为 1层
        self.epoch_num = 50 # 训练迭代次数
        self.sample_num = 10 # 随机采样
        self.batch_size = 32 # batch size
        self.seq_length = 35 # seq length
        self.lr = 0.002 #learning rate
        self.path = "./LSTM/jaychou_lyrics.txt" # 歌词数据集
        self.prefix = ['分开', '战争中', '我想'] # 测试阶段，给定的前缀，我们用它来生成歌词
        self.pred_len = 50 # 预测的字符长度
        self.use_gpu = True
        
config = Config()

In [4]:
# 这里简单一些，我们直接用一个函数来作为迭代器生成训练样本
def getBatch(corpus_indices, batch_size, seq_length, config):
    data_len = len(corpus_indices)
    batch_len = data_len // config.batch_size
    corpus_indices = torch.LongTensor(corpus_indices)
    # 将训练数据的 size 变成 batch_size x seq_length
    indices = corpus_indices[0: batch_size * batch_len].view(batch_size, batch_len)
    for i in range(0, indices.size(1) - seq_length, seq_length):
        input_data = Variable(indices[:, i: i + seq_length])
        target_data = Variable(indices[:, (i + 1): (i + 1) + seq_length].contiguous())
        # use GPU to train the model
        if config.use_gpu:
            input_data = input_data.cuda()
            target_data = target_data.cuda()
        yield(input_data, target_data)

In [5]:
# 将当前的状态从计算图中分离，加快训练速度
def detach(states):
    return [state.detach() for state in states] 

In [6]:
# 定义 LSTM 模型
class lstm(nn.Module):
    # input: 
    # x: 尺寸为 batch_size * seq_length 矩阵。
    # hidden: 尺寸为 batch_size * hidden_dim 矩阵。
    # output:
    # out: 尺寸为 batch_size * vocab_size 矩阵。
    # h: 尺寸为 batch_size * hidden_dim 矩阵。
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super(lstm, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.init_weights()
        
    def forward(self, x, hidden):
        embeds = self.embed(x)
        
        out, hidden = self.rnn(embeds, hidden)

        out = out.contiguous().view(out.size(0)*out.size(1), -1) # out 的 size 变成 (batch_size*sequence_length, hidden_size)
        
        out = self.linear(out) # (batch_size*sequence_length, hidden_size) -> (batch_size*sequence_length, vocab_size)
        return out, hidden
    
    def init_weights(self):
        self.embed.weight = nn.init.xavier_uniform(self.embed.weight)
        self.linear.bias.data.fill_(0)
        self.linear.weight = nn.init.xavier_uniform(self.linear.weight)

## 注意事项

在模型的初始化参数中，我们看到了 num_layers 这个参数，表示的含义是我们设置多少层隐含层。通常我们设置一层，当然也可以根据自己的意愿设置多层。

当 num_layers = 2 时，如图所示：

![num_layers = 2](./LSTM/img/num_layers.png)


**在训练过程中，我们会使用一个叫 detach 的函数** 这个函数的主要作用就是将隐含状态从计算图中分离出来。

假设，我们现在训练到了第 $t+1$ 个 batch， 那么这次训练过程中的隐含层输入是第 $t$ 个batch的输出隐含层状态，为了让模型参数的梯度计算只依赖于当前的批量序列，从而减小每次迭代的计算开销，我们可以使用detach函数来将隐含状态从计算图分离出来。

如果上面的话太拗口，一句话简单解释 detach 的作用就是： **在 RNN 模型中，这样做可以加速运算。**


In [7]:
# 构建语料库
corpus = Corpus(config.path)
# 处理 data
corpus.get_data()
# 模型初始化
lstm = lstm(len(corpus), config.embed_size, config.hidden_size, config.num_layers)
# 使用 gpu
if config.use_gpu:
    lstm = lstm.cuda()

In [8]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=config.lr)

In [9]:
# 使用训练好的 LSTM 在给定前缀的前提下，自动生成歌词。
def predict(model, prefix, config, corpus):
    """
        model 是模型， prefix是前缀， config 是参数类， corpus是语料库类
    """
    state_h = Variable(torch.zeros(config.num_layers, 1, config.hidden_size)) # 起始的hidden status
    state_c = Variable(torch.zeros(config.num_layers, 1, config.hidden_size)) # 起始的cell status
    
    # use gpu
    if config.use_gpu:
        state_h = state_h.cuda()
        state_c = state_c.cuda()
    # become a tuple
    state = (state_h, state_c)
    output = [corpus.char2id[prefix[0]]]
    for i in range(config.pred_len + len(prefix)):
        X = Variable(torch.LongTensor(output)).unsqueeze(0)
        # use gpu
        if config.use_gpu:
            X = X.cuda()
        Y, state = model(X, state)
        # 我们将结果变成概率，选择其中概率最大的作为预测下一个字符
        prob = Y.data[0].exp()
        word_id = torch.multinomial(prob, 1)[0]
        if i < len(prefix) - 1:
            next_char = corpus.char2id[prefix[i+1]]
        else:
            next_char = int(word_id)
        output.append(next_char)
    print("".join([corpus.id2char[id] for id in output]))
    return 

In [10]:
# 开始训练
for epoch in range(config.epoch_num):
    # 由于使用的是 lstm，我们初始化 hidden status 和 cell
    state_h = Variable(torch.zeros(config.num_layers, config.batch_size, config.hidden_size)) # 起始的hidden status
    state_c = Variable(torch.zeros(config.num_layers, config.batch_size, config.hidden_size)) # 起始的cell status
    # use gpu
    if config.use_gpu:
        state_h = state_h.cuda()
        state_c = state_c.cuda()
    
    hidden = (state_h, state_c)
    
    train_loss = [] # 训练的总误差
    
    for i,batch in enumerate(getBatch(corpus.corpus_indices, config.batch_size, config.seq_length, config)):
        inputs, targets = batch
        # Forward + Backward + Optimize
        lstm.zero_grad()
        hidden = detach(hidden)
        
        outputs, hidden = lstm(inputs, hidden)

        loss = criterion(outputs, targets.view(-1))
        train_loss.append(loss.data[0])
        loss.backward()
        torch.nn.utils.clip_grad_norm(lstm.parameters(), 0.5) # 梯度剪裁
        optimizer.step()
    # 采样，进行预测
    if epoch % config.sample_num == 0:
        print("Epoch %d. Perplexity %f" % (epoch, np.exp(np.mean(train_loss))))
        # 对给定的歌词开头，我们自动生成歌词
        for preseq in config.prefix:
            predict(lstm, preseq, config, corpus)

Epoch 0. Perplexity 536.237761
分开迎天过爱脚的半正去 寒l语  过梦文状素太上草的在在里衡与事安照 飘宗 西去外常的忆于活过功差的斯烧弹
战争中的的夫一南的弟猴清觉喜瑚掬墙走过朝哟你颜标不 贴马身功在驚杰 清狗朧像慰　幕这钱喘一帮 为夫摇运永灰 
我想哦想泪绪 咿无  婆一   花忆加持你的异专的你太 要飘统給到说一  無 的广四  双不喜的 永守 溪
Epoch 10. Perplexity 5.840660
分开觉纵手不回为忘历里 在旧无都的分下在好发在夕张里坠在来面看睡最 就来开手遍崖的地手 游开还心累的开道不
战争中光的不檐 疤默入` 兮杰度恰林  色腰候运瞑片蛮水叶a跑 e间 作伤声婪 之邦 腰谢兽 院巷 腰北情式
我想不舍  要故 有的爱生在的诗娘静完的还会 写在开朝全没来环  那们就不的绿说知试  们脸们的的们就在已
Epoch 20. Perplexity 1.176746
分开寸手来着到手才喜旧 穿生随明关开的变开你带接镜脸的的绪身 跑开开在情面有感命 开去水念说爱拳长 的去不
战争中落头 诉绪制静有南容水许落契 剧上国子 伤指在白场 洁具香也下下落  绪绪一朝入内 飘满头眼夜  檐制
我想给开  掉知在的面们 陪会要知的 没变会在只知怎没决相  会不踏一接一用们ㄟ说说开数 的的生只们们更睡
Epoch 30. Perplexity 1.028150
分开记难的新新候去 来身手示不于打应为关手 降里的的爱海  开手在飘手上留的在头方动头来下想来 想慢不长一
战争中答制 腰绪速继量 张字头里在眼平  慈持中支子 意命鱼夜续间命支蔓  兴头拍蓝吼照失归的 作角 的上丽
我想是睡 的将想只要甘女可的代  也也不有有们的的们不  不知再有 受轻的 默会里的的们 的轻们的的们  
Epoch 40. Perplexity 1.019534
分开下 的也生上回的  绪开开的可开就动手手将心套 美来开猜不可来手 的想下 可开喜也还去定来笔的的想  
战争中a色 迹色里夜的道绪声  视制中腰毛腰 声字过月 入命 虹露聆杀空 色牙中卡的球头  烟足林焚 灯惯易
我想睡  不拥再 一再一的 一变不变在  用不全有们们 睡不在  轻就的知牵 去们也都不永损就 的已那 变


这里只是简单的训练了 50 个 epoch，多次训练，可以看得出来，他们勉强有点... 像个句子了，尽管意思还是很模糊。